<a href="https://colab.research.google.com/github/Paymanshus/Sentiment-Based-Stock-Prediction/blob/master/Notebooks/Stock_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install backtrader
!pip install transformers

     |████████████████████████████████| 419kB 7.1MB/s 
     |████████████████████████████████| 1.5MB 7.4MB/s 
     |████████████████████████████████| 2.9MB 18.5MB/s 
     |████████████████████████████████| 890kB 23.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=5964fa84ac14ca9e7466c833a0bd439a929f081caefbfbb25d01526b311a6daa
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import nltk
nltk.download('vader_lexicon')

from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/MyDrive/finBERT/saved_text.csv')

In [ ]:
df

,Unnamed: 0,0,1
0,0,activision.txt,just what can just 1 to 7 Barbie cycle for jun...
1,1,cramer1.txt,mobile can sorry can you just crossed. WhatsAp...
2,2,market_correct.txt,because you bring a unique perspective window ...
3,3,papajohn.txt,hundred thousand restaurants close to the amou...
4,4,skillz.txt,Paradise in paradise today. spark. company wil...
5,5,tesla_29_3_20.txt,performance based on the rate of yesterday for...
6,6,tesla_3_12_20.txt,please. dcf valuation estimate 15 units in 201...
7,7,tesla_5_5_20.txt,NaN
8,8,WSupdate.txt,it was all our open this morning the diagonals...


In [ ]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop(df.columns[0], axis=1, inplace=True)
df.columns = ['file_name', 'text']

In [ ]:
df

,file_name,text
0,activision.txt,just what can just 1 to 7 Barbie cycle for jun...
1,cramer1.txt,mobile can sorry can you just crossed. WhatsAp...
2,market_correct.txt,because you bring a unique perspective window ...
3,papajohn.txt,hundred thousand restaurants close to the amou...
4,skillz.txt,Paradise in paradise today. spark. company wil...
5,tesla_29_3_20.txt,performance based on the rate of yesterday for...
6,tesla_3_12_20.txt,please. dcf valuation estimate 15 units in 201...
7,WSupdate.txt,it was all our open this morning the diagonals...


In [ ]:
dates = pd.read_csv('drive/MyDrive/finBERT/dates.csv')
dates.columns = ['file_name', 'Date']
dates

,file_name,date
0,tesla_29_3_20.txt,29_3_20
1,tesla_3_12_20.txt,3_12_20
2,tesla_5_5_20.txt,5_5_20


In [ ]:
df_final = pd.merge(df, dates, how='outer', on='file_name')

In [ ]:
df_final.dropna(inplace=True)
df_final.reset_index(drop=True, inplace=True)

In [ ]:
df_final

,file_name,text,date
0,tesla_29_3_20.txt,performance based on the rate of yesterday for...,29_3_20
1,tesla_3_12_20.txt,please. dcf valuation estimate 15 units in 201...,3_12_20


# Vader

Expected Results from Vader not optimal, need to perform analysis based on financial training data while Vader uses generic Wikipedia data etc.

In [ ]:
vader = SentimentIntensityAnalyzer()

In [ ]:
sentiments = {}

In [ ]:
vader.polarity_scores(df['text'][0])

{'compound': 0.9705, 'neg': 0.019, 'neu': 0.865, 'pos': 0.117}

In [ ]:
for i in range(df.shape[0]):
  sentiments[df['text'][i]] = vader.polarity_scores(df['text'][i])
sentiments

{'Paradise in paradise today. spark. company will be speech. ringmobi is per patient is to pay the competition is there. today we came up with you teams in the world where things we refer to it as the application of them everything from Bol games like we did that which is the most acceptable upon our things like education and exercise location. not too shabby this is this is the moment. covid-19 35% foreignchi. 20 minut now years this year. tecno mobile. tecno in1 Pro. on average tip of penis at higher than at higher than the industry what is. sports people longer. human and opportunity to compete in the favourite team is eating out. market 16% everything for joining us today. ': {'compound': 0.9601,
  'neg': 0.01,
  'neu': 0.838,
  'pos': 0.152},
 "because you bring a unique perspective window in the whether you're in the market for short term and long term is used as it was yesterday thinking about lightning up across your portfolio in fact you if done but I think you are coming yest

In [ ]:
comp_list = []

for i in range(len(sentiments)):
  comp_list.append(sentiments[list(sentiments)[i]]['compound'])

comp_list

[0.9705, 0.7096, 0.0, 0.9764, 0.9601, 0.5423, -0.1027, 0.0]

In [44]:
sent_list = ['positive' if i>0 else 'negative' if i<0 else 'neutral' for i in comp_list]
sent_list

['positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'negative',
 'neutral']

# FinBERT

Using FinBERT which is a BERT model pretrained on a Financial Lexicon and further trained using Supervised Learning on a Financial Sentiment Word List(Trained on Financial PhraseBank: https://www.kaggle.com/ankurzing/sentiment-analysis-for-financial-news?select=FinancialPhraseBank)

In [49]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('drive/MyDrive/finBERT/pytorch_model.bin', config='drive/MyDrive/finBERT/config.json', num_labels=3, from_pt=True)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [50]:
label_list = ['positive', 'negative', 'neutral']

In [51]:
def finbert_predict(text):
  inputs = tokenizer(text, return_tensors="tf", max_length=512, truncation=True)
  inputs["labels"] = tf.reshape(tf.constant(1), (-1, 1)) # Batch size 1
  outputs = model(inputs)

  sent = label_list[tf.argmax(outputs[1].numpy()[0]).numpy()]
  # print(sent)

  return sent

In [52]:
results = pd.DataFrame(df['text'].apply(finbert_predict))
results.columns = ['sentiment']
results

,sentiment
0,neutral
1,neutral
2,neutral
3,neutral
4,neutral
5,neutral
6,neutral
7,neutral


In [53]:
results['sentiment'] = results['sentiment'].replace({'neutral': 0, 'positive': 1, 'negative':-1})
results

,sentiment
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0


In [54]:
results_final = pd.DataFrame(df_final['text'].apply(finbert_predict))
results_final.columns = ['sentiment']
results_final

,sentiment
0,neutral
1,neutral


In [58]:
results_final = results_final.merge(df_final, how='inner', left_index=True, right_index=True)
results_final

,sentiment,file_name,text,date
0,neutral,tesla_29_3_20.txt,performance based on the rate of yesterday for...,29_3_20
1,neutral,tesla_3_12_20.txt,please. dcf valuation estimate 15 units in 201...,3_12_20


# Backtesting

In [45]:
tsla = pd.read_csv('drive/MyDrive/finBERT/TSLA.csv')

In [46]:
tsla.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-12-23,82.356003,84.402000,82.000000,83.844002,83.844002,66598000
1,2019-12-24,83.671997,85.094002,82.538002,85.050003,85.050003,40273500
2,2019-12-26,85.582001,86.695999,85.269997,86.188004,86.188004,53169500
3,2019-12-27,87.000000,87.061996,85.222000,86.075996,86.075996,49728500
4,2019-12-30,85.758003,85.800003,81.851997,82.940002,82.940002,62932000


In [47]:
tsla['Date']=pd.to_datetime(tsla['Date'])
tsla.set_index('Date', inplace=True)
tsla

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-12-23,82.356003,84.402000,82.000000,83.844002,83.844002,66598000
2019-12-24,83.671997,85.094002,82.538002,85.050003,85.050003,40273500
2019-12-26,85.582001,86.695999,85.269997,86.188004,86.188004,53169500
2019-12-27,87.000000,87.061996,85.222000,86.075996,86.075996,49728500
2019-12-30,85.758003,85.800003,81.851997,82.940002,82.940002,62932000
...,...,...,...,...,...,...
2020-12-14,619.000000,642.750000,610.200012,639.830017,639.830017,52040600
2020-12-15,643.280029,646.900024,623.799988,633.250000,633.250000,45223600
2020-12-16,628.229980,632.500000,605.000000,622.770020,622.770020,42095800


##Strategy 
If sentiment is found to be Positive -> Buy    
If sentiment is found to be Negative -> Sell

In [59]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt


# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('exitbars', 0),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.datasent = self.datas[0].sent

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.datasent[0] >2:
              # BUY, BUY, BUY!!! (with default parameters)
              self.log('BUY CREATE, %.2f' % self.dataclose[0])

              # Keep track of the created order to avoid a 2nd order
              self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + self.params.exitbars):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [60]:
class PandasSent(bt.feeds.PandasData):
  lines=(('sentiment'),)
  params=(('sentiment',-1),)

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Create a Data Feed
data = bt.feeds.PandasData(dataname=combined_df)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=100)

# Set the commission - 0.1% ... divide by 100 to remove the %
cerebro.broker.setcommission(commission=0.001)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
cerebro.plot(iplot=False)